# Noisy 10-Qubit Adaptive QNN Experiment
## Testing with Realistic NISQ Error Rates

**Experiment Overview:**
- Demonstrates adaptive QNN under realistic quantum noise
- NISQ error rates: 1% single-qubit, 1.5% two-qubit, 2% measurement
- 10 qubits, 30 epochs

## pip Installation and Environment

In [ ]:
# Check environment
try:
    import google.colab
    IN_COLAB = True
    print("Running on Google Colab")
    
    # GPU check
    !nvidia-smi
    
    # Install packages
    !pip install -q pennylane pennylane-lightning-gpu
    !pip install -q scikit-learn matplotlib seaborn
except:
    IN_COLAB = False
    print("Running locally")

print("\n✓ Setup complete!")

In [ ]:
import numpy as np
import pennylane as qml
from sklearn.datasets import load_wine, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from collections import defaultdict
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✓ Imports complete")

## Configuration with Realistic Noise

In [ ]:
class NoisyConfig:
    """Configuration with realistic NISQ noise parameters"""
    
    # Quantum system
    N_QUBITS = 10
    N_LAYERS = 2
    
    # Training
    LEARNING_RATE = 0.01
    N_EPOCHS = 30
    BATCH_SIZE = 16
    
    # Realistic NISQ error rates (IBM Quantum / Google Sycamore)
    NOISE_PARAMS = {
        'single_qubit_error': 0.01,      # 1% per gate
        'two_qubit_error': 0.015,        # 1.5% per gate
        'measurement_error': 0.02,       # 2% per measurement
        'description': 'Realistic NISQ (IBM/Google hardware)'
    }
    
    # Adaptive strategy
    STRATEGY = 'ucb'
    UCB_C = 2.0
    
    # Dataset
    DATASET = 'wine'
    TEST_SIZE = 0.3
    RANDOM_STATE = 42
    
    # Device - try GPU
    try:
        test_dev = qml.device('lightning.gpu', wires=2)
        DEVICE_NAME = 'lightning.gpu'
        print("✓ GPU acceleration available")
    except:
        DEVICE_NAME = 'default.mixed'  # For noise simulation
        print("Using CPU with mixed-state simulator")
    
    # Output
    VERBOSE = True
    PLOT_RESULTS = True
    SAVE_RESULTS = True
    OUTPUT_DIR = Path('results/noisy_10q')

config = NoisyConfig()
config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"\nConfiguration:")
print(f"  Qubits: {config.N_QUBITS}")
print(f"  Noise Model: {config.NOISE_PARAMS['description']}")
print(f"    Single-qubit error: {config.NOISE_PARAMS['single_qubit_error']*100}%")
print(f"    Two-qubit error: {config.NOISE_PARAMS['two_qubit_error']*100}%")
print(f"    Measurement error: {config.NOISE_PARAMS['measurement_error']*100}%")
print(f"  Epochs: {config.N_EPOCHS}")
print(f"  Strategy: {config.STRATEGY}")
print(f"  Dataset: {config.DATASET}")

## Helper Classes (Data, Features, Encoding)

In [ ]:
class DataLoader:
    @staticmethod
    def load_dataset(name='wine'):
        datasets = {'iris': load_iris, 'wine': load_wine}
        data = datasets[name]()
        X, y = data.data, data.target
        mask = y < 2
        return X[mask], y[mask], data.feature_names
    
    @staticmethod
    def preprocess(X_train, X_test, y_train, y_test):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        return X_train, X_test, y_train, y_test

class FeatureSelector:
    @staticmethod
    def pca(X_train, X_test, n_components):
        pca = PCA(n_components=n_components)
        return pca.fit_transform(X_train), pca.transform(X_test)
    
    @staticmethod
    def correlation(X_train, X_test, y_train, n_features):
        correlations = np.array([np.corrcoef(X_train[:, i], y_train)[0, 1] 
                                for i in range(X_train.shape[1])])
        top_indices = np.argsort(np.abs(correlations))[-n_features:]
        return X_train[:, top_indices], X_test[:, top_indices]
    
    @staticmethod
    def mutual_info(X_train, X_test, y_train, n_features):
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42)
        top_indices = np.argsort(mi_scores)[-n_features:]
        return X_train[:, top_indices], X_test[:, top_indices]
    
    @staticmethod
    def random_selection(X_train, X_test, n_features):
        np.random.seed(42)
        indices = np.random.choice(X_train.shape[1], n_features, replace=False)
        return X_train[:, indices], X_test[:, indices]

class QuantumEncoder:
    @staticmethod
    def angle_encoding(features, wires):
        for i, wire in enumerate(wires):
            if i < len(features):
                qml.RY(features[i], wires=wire)
    
    @staticmethod
    def amplitude_encoding(features, wires):
        n_amplitudes = 2 ** len(wires)
        padded = np.zeros(n_amplitudes)
        padded[:len(features)] = features
        normalized = padded / np.linalg.norm(padded) if np.linalg.norm(padded) > 0 else padded
        qml.AmplitudeEmbedding(normalized, wires=wires, normalize=True)

print("✓ Helper classes defined")

## Noisy Quantum Neural Network

In [ ]:
class NoisyQuantumNeuralNetwork:
    """QNN with realistic NISQ noise model"""
    
    def __init__(self, n_qubits, n_layers, encoding_method='angle', noise_params=None):
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.encoding_method = encoding_method
        self.noise_params = noise_params or config.NOISE_PARAMS
        
        # Device - use mixed state for noise
        try:
            # Try GPU first
            self.dev = qml.device('lightning.gpu', wires=n_qubits)
        except:
            # Fall back to mixed state simulator for noise
            self.dev = qml.device('default.mixed', wires=n_qubits)
        
        # Parameters
        self.n_params = n_layers * n_qubits * 3
        self.params = np.random.randn(self.n_params) * 0.1
        
        # Metrics
        self.gate_count = 0
        self.circuit_depth = 0
        
        # Create circuit
        self.qnode = qml.QNode(self._circuit, self.dev)
    
    def _circuit(self, features, params):
        """Quantum circuit with noise"""
        wires = range(self.n_qubits)
        self.gate_count = 0
        
        # Data encoding
        if self.encoding_method == 'angle':
            QuantumEncoder.angle_encoding(features, wires)
            self.gate_count += len(features)
        elif self.encoding_method == 'amplitude':
            QuantumEncoder.amplitude_encoding(features, wires)
            self.gate_count += 1
        
        # Parameterized layers with noise
        for layer in range(self.n_layers):
            # Single-qubit rotations
            for i in range(self.n_qubits):
                idx = layer * self.n_qubits * 3 + i * 3
                qml.RX(params[idx], wires=i)
                qml.RY(params[idx + 1], wires=i)
                qml.RZ(params[idx + 2], wires=i)
                self.gate_count += 3
                
                # Apply single-qubit noise
                if self.noise_params['single_qubit_error'] > 0:
                    qml.DepolarizingChannel(self.noise_params['single_qubit_error'], wires=i)
            
            # Entangling layer
            for i in range(self.n_qubits):
                qml.CNOT(wires=[i, (i + 1) % self.n_qubits])
                self.gate_count += 1
                
                # Apply two-qubit noise
                if self.noise_params['two_qubit_error'] > 0:
                    qml.DepolarizingChannel(self.noise_params['two_qubit_error'], wires=i)
                    qml.DepolarizingChannel(self.noise_params['two_qubit_error'], 
                                           wires=(i + 1) % self.n_qubits)
        
        self.circuit_depth = self.n_layers * 4 + 1
        
        # Measurement with noise
        if self.noise_params['measurement_error'] > 0:
            qml.DepolarizingChannel(self.noise_params['measurement_error'], wires=0)
        
        return qml.expval(qml.PauliZ(0))
    
    def predict_single(self, features):
        output = self.qnode(features, self.params)
        return 1 if output > 0 else 0
    
    def predict_batch(self, X):
        return np.array([self.predict_single(x) for x in X])
    
    def train_step(self, X_batch, y_batch, learning_rate):
        def loss_fn(params):
            predictions = np.array([self.qnode(x, params) for x in X_batch])
            targets = 2 * y_batch - 1
            return np.mean((predictions - targets) ** 2)
        
        grad_fn = qml.grad(loss_fn)
        gradients = grad_fn(self.params)
        self.params -= learning_rate * gradients

print("✓ NoisyQuantumNeuralNetwork defined")

## Adaptive Controller

In [ ]:
class AdaptiveController:
    """UCB-based adaptive configuration selection"""
    
    def __init__(self, configurations, strategy='ucb'):
        self.configurations = configurations
        self.strategy = strategy
        self.rewards = {cfg: [] for cfg in configurations}
        self.selection_counts = {cfg: 0 for cfg in configurations}
        self.total_selections = 0
        self.selection_history = []
        self.reward_history = []
        self.ucb_c = config.UCB_C
    
    def select_configuration(self):
        # Ensure each tried once
        if self.total_selections < len(self.configurations):
            config_name = self.configurations[self.total_selections]
            self.selection_history.append(config_name)
            return config_name
        
        # UCB selection
        ucb_values = {}
        for cfg in self.configurations:
            if len(self.rewards[cfg]) == 0:
                ucb_values[cfg] = float('inf')
            else:
                mean_reward = np.mean(self.rewards[cfg])
                exploration = self.ucb_c * np.sqrt(
                    np.log(self.total_selections) / self.selection_counts[cfg]
                )
                ucb_values[cfg] = mean_reward + exploration
        
        selected = max(ucb_values, key=ucb_values.get)
        self.selection_history.append(selected)
        return selected
    
    def update(self, configuration, reward):
        self.rewards[configuration].append(reward)
        self.selection_counts[configuration] += 1
        self.total_selections += 1
        self.reward_history.append(reward)
    
    def get_best_configuration(self):
        mean_rewards = {
            cfg: np.mean(rewards) if rewards else 0
            for cfg, rewards in self.rewards.items()
        }
        return max(mean_rewards, key=mean_rewards.get)
    
    def get_statistics(self):
        stats = []
        for cfg in self.configurations:
            if self.rewards[cfg]:
                stats.append({
                    'configuration': cfg,
                    'mean_reward': np.mean(self.rewards[cfg]),
                    'std_reward': np.std(self.rewards[cfg]),
                    'count': len(self.rewards[cfg])
                })
        return sorted(stats, key=lambda x: x['mean_reward'], reverse=True)

print("✓ AdaptiveController defined")

## Main Training Pipeline

In [ ]:
class NoisyAdaptiveQNNTrainer:
    """Main training pipeline with noise"""
    
    def __init__(self):
        # Load data
        print("Loading dataset...")
        X, y, _ = DataLoader.load_dataset(config.DATASET)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=config.TEST_SIZE, random_state=config.RANDOM_STATE
        )
        X_train, X_test, y_train, y_test = DataLoader.preprocess(
            X_train, X_test, y_train, y_test
        )
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        print(f"Dataset: {config.DATASET}")
        print(f"Training: {len(X_train)}, Test: {len(X_test)}")
        
        # Configurations
        feature_methods = ['PCA', 'Correlation', 'MutualInfo', 'Random']
        encoding_methods = ['angle', 'amplitude']
        self.configurations = [
            f"{fm}+{em}" for fm in feature_methods for em in encoding_methods
        ]
        
        # Controller
        self.controller = AdaptiveController(self.configurations, config.STRATEGY)
        
        # Tracking
        self.training_history = {
            'train_acc': [],
            'val_acc': [],
            'config': [],
            'epoch_time': [],
            'gate_count': [],
            'circuit_depth': []
        }
        
        self.start_time = time.time()
    
    def apply_feature_selection(self, method):
        if method == 'PCA':
            return FeatureSelector.pca(self.X_train, self.X_test, config.N_QUBITS)
        elif method == 'Correlation':
            return FeatureSelector.correlation(self.X_train, self.X_test, self.y_train, config.N_QUBITS)
        elif method == 'MutualInfo':
            return FeatureSelector.mutual_info(self.X_train, self.X_test, self.y_train, config.N_QUBITS)
        elif method == 'Random':
            return FeatureSelector.random_selection(self.X_train, self.X_test, config.N_QUBITS)
    
    def train_epoch(self, qnn, X_train, X_test, y_train, y_test):
        epoch_start = time.time()
        
        # Training
        n_samples = len(X_train)
        indices = np.random.permutation(n_samples)
        
        for i in range(0, n_samples, config.BATCH_SIZE):
            batch_indices = indices[i:i + config.BATCH_SIZE]
            qnn.train_step(X_train[batch_indices], y_train[batch_indices], config.LEARNING_RATE)
        
        # Evaluation
        train_pred = qnn.predict_batch(X_train)
        train_acc = np.mean(train_pred == y_train)
        
        val_pred = qnn.predict_batch(X_test)
        val_acc = np.mean(val_pred == y_test)
        
        epoch_time = time.time() - epoch_start
        
        return train_acc, val_acc, epoch_time, qnn.gate_count, qnn.circuit_depth
    
    def run(self):
        print(f"\n{'='*70}")
        print("STARTING NOISY ADAPTIVE QNN TRAINING")
        print(f"{'='*70}")
        print(f"Noise: {config.NOISE_PARAMS['description']}")
        print(f"Epochs: {config.N_EPOCHS}")
        print(f"{'='*70}\n")
        
        for epoch in range(config.N_EPOCHS):
            # Select configuration
            config_name = self.controller.select_configuration()
            feature_method, encoding_method = config_name.split('+')
            
            # Feature selection
            X_train_reduced, X_test_reduced = self.apply_feature_selection(feature_method)
            
            # Create noisy QNN
            qnn = NoisyQuantumNeuralNetwork(
                config.N_QUBITS, config.N_LAYERS, encoding_method, config.NOISE_PARAMS
            )
            
            # Train
            train_acc, val_acc, epoch_time, gate_count, circuit_depth = self.train_epoch(
                qnn, X_train_reduced, X_test_reduced, self.y_train, self.y_test
            )
            
            # Update controller
            self.controller.update(config_name, val_acc)
            
            # Track
            self.training_history['train_acc'].append(train_acc)
            self.training_history['val_acc'].append(val_acc)
            self.training_history['config'].append(config_name)
            self.training_history['epoch_time'].append(epoch_time)
            self.training_history['gate_count'].append(gate_count)
            self.training_history['circuit_depth'].append(circuit_depth)
            
            # Progress
            if config.VERBOSE:
                best_config = self.controller.get_best_configuration()
                best_reward = np.mean(self.controller.rewards[best_config]) if self.controller.rewards[best_config] else 0
                
                print(f"Epoch {epoch+1:3d}/{config.N_EPOCHS} | "
                      f"Config: {config_name:25s} | "
                      f"Train: {train_acc:.3f} | "
                      f"Val: {val_acc:.3f} | "
                      f"Best: {best_config:20s} ({best_reward:.3f}) | "
                      f"Time: {epoch_time:.1f}s")
        
        self.total_time = time.time() - self.start_time
        self.generate_report()
        
        if config.SAVE_RESULTS:
            self.save_results()
        
        if config.PLOT_RESULTS:
            self.plot_results()
    
    def generate_report(self):
        print(f"\n{'='*70}")
        print("NOISY QNN EXPERIMENT REPORT")
        print(f"{'='*70}")
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        print(f"\nNoise Model:")
        print(f"  {config.NOISE_PARAMS['description']}")
        print(f"  Single-qubit: {config.NOISE_PARAMS['single_qubit_error']*100}%")
        print(f"  Two-qubit: {config.NOISE_PARAMS['two_qubit_error']*100}%")
        print(f"  Measurement: {config.NOISE_PARAMS['measurement_error']*100}%")
        
        print(f"\nTraining Summary:")
        print(f"  Total Epochs: {config.N_EPOCHS}")
        print(f"  Total Time: {self.total_time:.2f}s")
        print(f"  Avg Time/Epoch: {self.total_time/config.N_EPOCHS:.2f}s")
        
        best_config = self.controller.get_best_configuration()
        best_rewards = self.controller.rewards[best_config]
        
        print(f"\nBest Configuration (under noise):")
        print(f"  Config: {best_config}")
        print(f"  Mean Reward: {np.mean(best_rewards):.4f}")
        print(f"  Std Dev: {np.std(best_rewards):.4f}")
        print(f"  Times Selected: {len(best_rewards)}")
        
        print(f"\nAll Configurations:")
        print(f"{'Configuration':<25s} {'Mean':<10s} {'Std':<10s} {'Count':<8s}")
        print("-" * 70)
        
        stats = self.controller.get_statistics()
        for stat in stats:
            print(f"{stat['configuration']:<25s} "
                  f"{stat['mean_reward']:<10.4f} "
                  f"{stat['std_reward']:<10.4f} "
                  f"{stat['count']:<8d}")
        
        print(f"\n{'='*70}\n")
    
    def save_results(self):
        results = {
            'config': {
                'n_qubits': config.N_QUBITS,
                'n_layers': config.N_LAYERS,
                'n_epochs': config.N_EPOCHS,
                'noise_params': config.NOISE_PARAMS
            },
            'training_history': self.training_history,
            'controller_stats': self.controller.get_statistics(),
            'total_time': self.total_time
        }
        
        filename = config.OUTPUT_DIR / f'noisy_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
        with open(filename, 'w') as f:
            json.dump(results, f, indent=2)
        
        print(f"✓ Results saved: {filename}")
    
    def plot_results(self):
        fig = plt.figure(figsize=(18, 12))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # Plot 1: Training progress
        ax = fig.add_subplot(gs[0, :2])
        epochs = range(1, len(self.training_history['train_acc']) + 1)
        ax.plot(epochs, self.training_history['train_acc'], label='Train', linewidth=2, alpha=0.7)
        ax.plot(epochs, self.training_history['val_acc'], label='Validation', linewidth=2, alpha=0.7)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy')
        ax.set_title('Training Progress with Noise')
        ax.legend()
        ax.grid(alpha=0.3)
        
        # Plot 2: Config frequency
        ax = fig.add_subplot(gs[0, 2])
        config_counts = {cfg: self.controller.selection_counts[cfg] for cfg in self.configurations}
        configs = list(config_counts.keys())
        counts = list(config_counts.values())
        
        y_pos = np.arange(len(configs))
        ax.barh(y_pos, counts)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(configs, fontsize=9)
        ax.set_xlabel('Times Selected')
        ax.set_title('Config Selection')
        ax.grid(axis='x', alpha=0.3)
        
        # Plot 3: Reward over time
        ax = fig.add_subplot(gs[1, :2])
        ax.plot(self.controller.reward_history, linewidth=2, color='green', alpha=0.7)
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Reward (Val Accuracy)')
        ax.set_title('Reward Over Time')
        ax.grid(alpha=0.3)
        
        # Plot 4: Config timeline
        ax = fig.add_subplot(gs[1, 2])
        config_to_y = {cfg: i for i, cfg in enumerate(self.configurations)}
        
        for i, (cfg, reward) in enumerate(zip(self.controller.selection_history, 
                                              self.controller.reward_history)):
            ax.scatter(i, config_to_y[cfg], c=[reward], vmin=0, vmax=1, 
                      cmap='viridis', s=80, edgecolors='black', linewidth=0.5)
        
        ax.set_yticks(range(len(self.configurations)))
        ax.set_yticklabels(self.configurations, fontsize=8)
        ax.set_xlabel('Epoch')
        ax.set_title('Config Timeline')
        
        # Plot 5: Performance by config
        ax = fig.add_subplot(gs[2, :])
        stats = self.controller.get_statistics()
        configs = [s['configuration'] for s in stats]
        means = [s['mean_reward'] for s in stats]
        stds = [s['std_reward'] for s in stats]
        
        x_pos = np.arange(len(configs))
        ax.bar(x_pos, means, yerr=stds, capsize=5, alpha=0.7)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(configs, rotation=45, ha='right')
        ax.set_ylabel('Mean Validation Accuracy')
        ax.set_title('Performance by Configuration (with Noise)')
        ax.grid(axis='y', alpha=0.3)
        
        plt.suptitle('Noisy 10-Qubit Adaptive QNN Results', fontsize=16, fontweight='bold')
        
        filename = config.OUTPUT_DIR / f'noisy_plots_{datetime.now().strftime("%Y%m%d_%H%M%S")}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"✓ Plots saved: {filename}")
        
        plt.show()

print("✓ NoisyAdaptiveQNNTrainer defined")

## Run Experiment

In [ ]:
# Create and run trainer
trainer = NoisyAdaptiveQNNTrainer()
trainer.run()

## Download Results

In [ ]:
if IN_COLAB:
    from google.colab import files
    import shutil
    
    shutil.make_archive('noisy_10q_results', 'zip', config.OUTPUT_DIR)
    
    print("Downloading results...")
    files.download('noisy_10q_results.zip')
    print("✓ Download complete!")
else:
    print("Not on Colab - results saved locally")